# State-Machine States

### State Name : New Contract Creation (Yeni Sözleşme Oluşturma)

1. All of json copied without id's for process
2. Startdate is changed to currentdate

3. Changed startDate's endDates is also changed with active ones 

##### Pre-Execute Script

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, prev_state_response):
    # do workflow here
    return {"success": True, "goto": "END"}

##### Business Mapping

In [1]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}

from datetime import datetime
import json 
def business_mapping(machine, state, request_data, context, prev_state_response, current_state_response):
    #Not recursive 
    current_date = datetime.now().strftime('%Y%m%d%H%M%S')
    
    copied_json = {}
    stack = [(copied_json, context)]

    while stack:
        current_copied, current_original = stack.pop()

        for key, value in current_original.items():
            if key != 'id':
                if isinstance(value, list):
                    new_list = []
                    for item in value:
                        if isinstance(item, dict):
                            new_item = {}
                            new_list.append(new_item)
                            stack.append((new_item, item))
                        else:
                            new_list.append(item)
                    current_copied[key] = new_list
                elif isinstance(value, dict):
                    new_dict = {}
                    current_copied[key] = new_dict
                    stack.append((new_dict, value))
                else:
                    current_copied[key] = value

    # Tüm değerlerin tarihlerini ayarla
    def set_dates(data_dict):
        if isinstance(data_dict,dict):
            if data_dict.get('value') is None and 'values' in data_dict and data_dict['values']:
                data_dict['values'][0]['startDate'] = current_date
                data_dict['values'][0]['endDate'] = 99991231235959
            elif data_dict.get('value') is not None:
                data_dict['startDate'] = current_date
                data_dict['endDate'] = 99991231235959
        
    for key, value in copied_json.items():
        if isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    set_dates(item)
        elif isinstance(value, dict):
            set_dates(value)

    return copied_json  

In [2]:
with open ('state7_input.json') as f:
    context = json.load(f)

In [3]:
context

{'id': 21408,
 'assetDefinitionCode': 'LicensedProducerContract',
 'assetName': None,
 'authorityGroup': 'string',
 'assets': None,
 'attributes': [{'id': None,
   'attribute': {'alias': None, 'attributeCode': 'ReactiveExemption'},
   'value': None,
   'startDate': None,
   'endDate': None,
   'valueDesc': None,
   'values': [{'id': 13512,
     'attribute': None,
     'value': False,
     'startDate': 20241001000000,
     'endDate': 99991231235959,
     'valueDesc': None,
     'values': None}]},
  {'id': 16057,
   'attribute': {'alias': None, 'attributeCode': 'ContractNumber'},
   'value': 10000446,
   'startDate': None,
   'endDate': None,
   'valueDesc': None,
   'values': None},
  {'id': 13071,
   'attribute': {'alias': None,
    'attributeCode': 'ContractCancellationRequestDate'},
   'value': 99991231235959,
   'startDate': None,
   'endDate': None,
   'valueDesc': None,
   'values': None},
  {'id': 13070,
   'attribute': {'alias': None, 'attributeCode': 'OldSystemContractBeginDate

In [6]:
from datetime import datetime

def copy_json_without_ids(original_json):
    #recursive one
    current_date = datetime.now().strftime('%Y%m%d%H%M%S')
    
    copied_json = {}

    for key, value in original_json.items():
        if key != 'id':
            if isinstance(value, list):
                copied_json[key] = []
                for item in value:
                    if isinstance(item, dict):
                        copied_item = copy_json_without_ids(item)                    
                        if copied_item.get('value') is None and 'values' in copied_item and copied_item['values']:
                            copied_item['values'][0]['startDate'] = current_date
                            copied_item['values'][0]['endDate'] = 99991231235959  
                        elif copied_item.get('value') is not None:
                            copied_item['startDate'] = current_date
                            copied_item['endDate'] = 99991231235959  
                        copied_json[key].append(copied_item)
                    else:
                        copied_json[key].append(item)
            elif isinstance(value, dict):
                copied_json[key] = copy_json_without_ids(value)
            else:
                copied_json[key] = value

    return copied_json